In [1]:
%matplotlib widget
import caesar
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as pkl

/Users/ondrea/anaconda3/lib/python3.8/_collections_abc.py:666: MatplotlibDeprecationWarning: The global colormaps dictionary is no longer considered public API.
  self[key]


 In this notebook I have several methods for cleaning and cutting galaxy data before determining the cause of quenching.

In [2]:
obj = caesar.load('/Users/ondrea/Downloads/caesar_296.hdf5') # 296=redshift0

yt : [INFO     ] 2021-09-01 15:00:54,842 Opening /Users/ondrea/Downloads/caesar_296.hdf5
yt : [INFO     ] 2021-09-01 15:00:55,074 Found 54135 halos
yt : [INFO     ] 2021-09-01 15:00:55,115 Found 6480 galaxies


In [3]:
sfr = np.array([gal.sfr for gal in obj.galaxies])   # I want to cut data by SFR and mass
stellar_masses = np.array([gal.masses['stellar'] for gal in obj.galaxies])

In [4]:
stellar_masses[stellar_masses == 0] = 1e-99  #for log
sfr[sfr == 0] = 1e-99

In [5]:
stellar_masses

array([1.05960459e+12, 8.26104807e+11, 7.95258585e+11, ...,
       2.64359616e+08, 2.61941984e+08, 2.61884624e+08])

In [6]:
#Make a col of specific star formation rate (SFR/gal stellar mass)*(1e9 Gyr/yr)
ssfr = (sfr/stellar_masses)*1e9 

Since the science question is "what is the cause of quenching in high mass galaxies?" I need to remove low mass ones. I'll start with log(M/Msun) = 9.5

I am also removing zero values for now for the sake of clustering algs

In [7]:
high_mass_idx = np.where((np.log10(stellar_masses) >= 9.5) & (ssfr >1e-4))

In [8]:
stellar_masses = np.log10(stellar_masses[high_mass_idx])
ssfr = np.log10(ssfr[high_mass_idx])

In [9]:
from sklearn.mixture import GaussianMixture

In [10]:
X_full = np.zeros([len(ssfr), 2])
for i in range(len(ssfr)):
    X_full[i,1] = ssfr[i]
    X_full[i,0] = stellar_masses[i]

In [11]:
X_full

array([[11.60380662, -1.02006806],
       [11.57514968, -1.18039787],
       [11.50953694, -1.65681067],
       ...,
       [ 9.50177249, -1.22522462],
       [ 9.5013506 , -0.51490805],
       [ 9.50107008, -0.28143188]])

In [12]:
gmm = GaussianMixture(n_components=2, covariance_type='diag',random_state=1).fit(X_full)
labels = gmm.predict(X_full)

In [13]:
ifig=2;plt.close(ifig);plt.figure(ifig)
#plt.scatter(gal_data.stellar_masses[labels == 1], np.log10(gal_data.ssfr[labels == 1]), s=4, alpha=0.4, c='b');
#plt.scatter(gal_data.stellar_masses[labels == 0], np.log10(gal_data.ssfr[labels == 0]), s=4, alpha=0.4, c='k');
plt.scatter(X_full[:, 0], X_full[:, 1], c=labels, s=2, cmap='viridis');
plt.xlabel('log stellar mass')
plt.ylabel('log ssfr')
plt.xlim(9.5,12);
plt.ylim(-4,1);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

I think a polynomial SVM classifier might give me something closer to what I want

In [14]:
from sklearn.svm import SVC
clf=SVC

In [45]:
#Make training and test data
np.random.seed(99)
idxs = np.random.randint(0, len(X_full)+1, size=20)
X = X_full[idxs]
y = np.array([1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0  ]) #1=dead

test_data = np.delete(X_full, idxs)

In [46]:
test_data

array([11.60380662, -1.02006806, 11.57514968, ..., -0.51490805,
        9.50107008, -0.28143188])

In [47]:
print(idxs[10:])

[  59 2022 1387  244 1239 1666 2240  652 1448 1044]


In [48]:
X

array([[10.32101425, -1.38400224],
       [10.06014744, -1.33661614],
       [ 9.86151065, -1.09592336],
       [ 9.84811848, -0.58141426],
       [10.09695896, -0.36213595],
       [ 9.74139521, -0.49420271],
       [11.0739395 , -0.93838782],
       [10.46921114, -2.18632914],
       [10.81558591, -1.49283952],
       [10.04891726, -1.25664136],
       [11.05795655, -1.69945654],
       [ 9.71394121, -1.23723785],
       [ 9.99624157, -1.10728138],
       [10.69126844, -0.94086886],
       [10.05076914, -0.8863065 ],
       [ 9.89076194, -1.75196462],
       [ 9.59296455, -0.47149927],
       [10.3145216 , -1.4781833 ],
       [ 9.97143684, -1.09168605],
       [10.11924229, -0.85633657]])

In [49]:
#ifig=3;plt.close(ifig);plt.figure(ifig)  #use this plot to plot and tag points in 'y' as dead or not dead
#plt.scatter(X_full[:, 0], X_full[:, 1],s=1)
#plt.scatter(X[19, 0], X[19, 1],s=10, c='k')

In [50]:
clf = SVC(kernel='rbf', random_state=1).fit(X, y=y)

In [51]:
decision_function = clf.decision_function(X)
support_vectors = clf.support_vectors_

In [52]:
print(decision_function)

[-0.79802381 -0.84493767 -0.91023665 -0.98593889 -0.97918626 -1.0140611
 -0.74750375 -0.66055085 -0.70668135 -0.8583126  -0.64090631 -0.91202568
 -0.88809509 -0.80553521 -0.91177167 -0.8103156  -1.03905926 -0.78531896
 -0.89414188 -0.905638  ]


In [53]:
ifig=2;plt.close(ifig);plt.figure(ifig)

# "hinge" is the standard SVM loss
clf = SVC(kernel='poly',degree=7, C=10,random_state=1).fit(X, y=y)
#clf = SVC(kernel='Linear', C=100,random_state=1).fit(X, y=y)
#clf = SVC(kernel='rbf', C=200,random_state=1).fit(X, y=y)
# obtain the support vectors through the decision function
decision_function = clf.decision_function(X)
support_vectors = clf.support_vectors_


plt.scatter(X_full[:, 0],X_full[:, 1],s=1, color='grey',zorder=1)
plt.scatter(X[:, 0], X[:, 1], c=y, s=30, edgecolors='k', cmap=sns.color_palette("Spectral", as_cmap=True))
ax = plt.gca()
xlim = (9.5,12)
ylim = (-4,1)
xx, yy = np.meshgrid(np.linspace(xlim[1], xlim[0], 50),
                     np.linspace(ylim[1], ylim[0], 50))
Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.scatter(support_vectors[:, 0], support_vectors[:, 1], s=100,
            linewidth=2, facecolors='none', edgecolors='k', alpha=0.8 )


plt.contour(xx, yy, Z, colors='k', levels=[-1, 0, 1], alpha=0.7,
            linestyles=['--', '-', '--'])

plt.xlabel('log stellar mass')
plt.ylabel('log ssfr')
plt.savefig('./polynomial_classifier.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
clf.decision_function(np.c_[xx.ravel(), yy.ravel()])

array([-0.75663734, -0.88539283, -1.01465067, ...,  5.9182527 ,
        5.79985255,  5.68134615])

In [56]:
pkl.dump(clf, open('poly_clf', 'wb'))